In [1]:
import meshio

In [2]:
mesh = meshio.read("ice-multiple-crevasse-adaptive.msh")
points, cell, cell_data, field_data = mesh.points, mesh.cells, mesh.cell_data, mesh.field_data

In [3]:
points[:,0].min(), points[:,1].min(), points[:,0].max()

(0.0, 0.0, 500.0)

In [4]:
meshio.write("ice-adaptive.xdmf", meshio.Mesh(
    points=points[:,0:2],
    cells={"triangle": cell['triangle']}))

In [50]:
cell_data

{'triangle': {'gmsh:physical': array([173, 173, 173, ..., 174, 174, 174])}}

In [51]:
meshio.write("triangle.xdmf", meshio.Mesh(
    points=points,
    cells={"triangle": cell['triangle']},
    cell_data={"triangle": {"marker": cell_data["triangle"]["gmsh:physical"]}}
))

# Demo

In [10]:
from dolfin import *

In [11]:
mesh_2d = Mesh()
with XDMFFile("mesh.xdmf") as infile:
    infile.read(mesh_2d)

In [12]:
mvc = MeshValueCollection("size_t", mesh_2d, 0)
with XDMFFile("triangle.xdmf") as infile:
    infile.read(mvc, "marker")

mf = cpp.mesh.MeshFunctionSizet(mesh_2d, mvc)

# mf.array()[mf.array()>20]=0

with XDMFFile("mf/mesh_function.xdmf") as xdmf_outfile:
    xdmf_outfile.write(mf)